In [1]:
import librosa
import librosa.display
import glob
from scipy.signal import butter, filtfilt, freqz
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import re
import platform
import subprocess as sp

In [2]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order)
    y = filtfilt(b, a, data)
    return np.array(y)

In [3]:
def plot_audio(data):
    plt.figure(figsize=(10, 4))
    # Variables for plotting
    n = len(data)           # total number of samples
    T = n / fs              # seconds
    t = np.linspace(0, T, n)
    plt.plot(t, data, linewidth=.2)
    plt.show()

In [4]:
def get_MFCC_per_folder(path, cutoff=400, order=6, top_db=20):
    wav_file_list = [f.path for f in os.scandir(path)]
    mfcc_dataset = {}
    for wav_file in wav_file_list:
        
        # Get identifier of the audio clip
        key = re.findall('\d+', wav_file)[0]
        sound, sampling_rate = librosa.load(wav_file)
        fs = sampling_rate   # sample rate, Hz
        
        # low-pass filter the sound by removing all frequencies
        # above the cutoff value (default=400 Hz)
        low_passed_sound = butter_lowpass_filter(sound, cutoff, fs, order)
        
        # Trim the silent parts of the speech signal
        low_passed_trimmed_sound, _ = librosa.effects.trim(low_passed_sound, top_db=top_db)
        
        # Calculate the MFCC values of the low-pass filtered and trimmed speech signal
        mfccs = librosa.feature.mfcc(y=low_passed_trimmed_sound, sr=sampling_rate, n_mfcc=12).tolist()
        mfcc_dataset[key] = mfccs
        
    return mfcc_dataset

In [5]:
def save_MFCC_as_json(MFCC_data, filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(MFCC_data, f, ensure_ascii=False, indent=4)
    return None

def set_path_seperator():
    path_sep = "/"
    if platform.system() == "Windows":
        path_sep = "\\"
    return path_sep


def get_paths_of_language(source_path, language_folder, sub_folders):
    languages = [f.name for f in os.scandir(source_path)]
    if not language_folder in languages:
        print('You do not have any data of this language!')
        return None
    else:
        paths = []
        for folder in sub_folders:
            if platform.system() == 'Windows':
                paths.append(source_path + "\\" + language_folder + "\\" + folder + "\\" + '\\')
            else:
                paths.append(source_path + "/" + language_folder + "/" + folder + "/")
        return paths
    
def convert_to_windows_path(path):
    return "\\".join(path.split('/'))

In [11]:
# IMPORTANT: in case you have downloaded the data in a different audio format (for example in .mp3)
# convert all audio files that you want to use into .wav files

paths_nl = get_paths_of_language('Downsampled', 'nl',\
                              ['train_wav_clips', 'test_wav_clips', 'validate_wav_clips'])

paths_en = get_paths_of_language('Downsampled', 'en',\
                              ['train_wav_clips', 'test_wav_clips', 'validate_wav_clips'])

paths_it = get_paths_of_language('Downsampled', 'it',\
                              ['train_wav_clips', 'test_wav_clips', 'validate_wav_clips'])

paths_es = get_paths_of_language('Downsampled', 'es',\
                              ['train_wav_clips', 'test_wav_clips', 'validate_wav_clips'])

# Variables needed for the filter
order = 6
cutoff = 400            # desired cutoff frequency of the filter, Hz

path_sep = set_path_seperator()

In [12]:
MFCC_train_set_nl = get_MFCC_per_folder(paths_nl[0])
MFCC_test_set_nl = get_MFCC_per_folder(paths_nl[1])
MFCC_validate_set_nl = get_MFCC_per_folder(paths_nl[2])

# saving to files for nl
language_folder_nl = paths_nl[0].split(path_sep)[1]
dst_folder_nl = "Downsampled" + path_sep + language_folder_nl + path_sep + 'MFCC_json_files' + path_sep

sp.run("rm -r " + dst_folder_nl, shell=True)
sp.run("mkdir " + dst_folder_nl, shell=True)

save_MFCC_as_json(MFCC_train_set_nl, dst_folder_nl + 'MFCC_train.json')
save_MFCC_as_json(MFCC_test_set_nl, dst_folder_nl + 'MFCC_test.json')
save_MFCC_as_json(MFCC_validate_set_nl, dst_folder_nl + 'MFCC_validate.json')

In [ ]:
MFCC_train_set_en = get_MFCC_per_folder(paths_en[0])
MFCC_test_set_en = get_MFCC_per_folder(paths_en[1])
MFCC_validate_set_en = get_MFCC_per_folder(paths_en[2])

# saving to files for en
language_folder_en = paths_en[0].split(path_sep)[1]
dst_folder_en = "Downsampled" + path_sep + language_folder_en + path_sep + 'MFCC_json_files' + path_sep

sp.run("rm -r " + dst_folder_en, shell=True)
sp.run("mkdir " + dst_folder_en, shell=True)

save_MFCC_as_json(MFCC_train_set_en, dst_folder_en + 'MFCC_train.json')
save_MFCC_as_json(MFCC_test_set_en, dst_folder_en + 'MFCC_test.json')
save_MFCC_as_json(MFCC_validate_set_en, dst_folder_en + 'MFCC_validate.json')

In [ ]:
MFCC_train_set_it = get_MFCC_per_folder(paths_it[0])
MFCC_test_set_it = get_MFCC_per_folder(paths_it[1])
MFCC_validate_set_it = get_MFCC_per_folder(paths_it[2])

# saving to files for it
language_folder_it = paths_it[0].split(path_sep)[1]
dst_folder_it = "Downsampled" + path_sep + language_folder_it + path_sep + 'MFCC_json_files' + path_sep

sp.run("rm -r " + dst_folder_it, shell=True)
sp.run("mkdir " + dst_folder_it, shell=True)

save_MFCC_as_json(MFCC_train_set_it, dst_folder_it + 'MFCC_train.json')
save_MFCC_as_json(MFCC_test_set_it, dst_folder_it + 'MFCC_test.json')
save_MFCC_as_json(MFCC_validate_set_it, dst_folder_it + 'MFCC_validate.json')

In [ ]:
MFCC_train_set_es = get_MFCC_per_folder(paths_es[0])
MFCC_test_set_es = get_MFCC_per_folder(paths_es[1])
MFCC_validate_set_es = get_MFCC_per_folder(paths_es[2])

# saving to files for es
language_folder_es = paths_es[0].split(path_sep)[1]
dst_folder_es = "Downsampled" + path_sep + language_folder_es + path_sep + 'MFCC_json_files' + path_sep

sp.run("rm -r " + dst_folder_es, shell=True)
sp.run("mkdir " + dst_folder_es, shell=True)

save_MFCC_as_json(MFCC_train_set_es, dst_folder_es + 'MFCC_train.json')
save_MFCC_as_json(MFCC_test_set_es, dst_folder_es + 'MFCC_test.json')
save_MFCC_as_json(MFCC_validate_set_es, dst_folder_es + 'MFCC_validate.json')